# Historical Variance

Let's see how we'd be calculating a covariance matrix of assets without the help of a factor model

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [1]:
import numpy as np
import pandas as pd
import time
import os
import quiz_helper
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

### data bundle

In [3]:
import os
import quiz_helper
from zipline.data import bundles

In [4]:
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..','data','module_4_quizzes_eod')
ingest_func = bundles.csvdir.csvdir_equities(['daily'], quiz_helper.EOD_BUNDLE_NAME)
bundles.register(quiz_helper.EOD_BUNDLE_NAME, ingest_func)
print('Data Registered')

Data Registered


### Build pipeline engine

In [5]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar

universe = AverageDollarVolume(window_length=120).top(500) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(quiz_helper.EOD_BUNDLE_NAME)
engine = quiz_helper.build_pipeline_engine(bundle_data, trading_calendar)

ValueError: no data for bundle 'm4-quiz-eod-quotemedia' on or before 2018-11-19 22:07:42.917486+00:00
maybe you need to run: $ zipline ingest -b m4-quiz-eod-quotemedia

### View Data¶
With the pipeline engine built, let's get the stocks at the end of the period in the universe we're using. We'll use these tickers to generate the returns data for the our risk model.

In [6]:
universe_end_date = pd.Timestamp('2016-01-05', tz='UTC')

universe_tickers = engine\
    .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date)\
    .index.get_level_values(1)\
    .values.tolist()
    
universe_tickers

NameError: name 'engine' is not defined

In [ ]:
len(universe_tickers)

In [ ]:
from zipline.data.data_portal import DataPortal

data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

## Get pricing data helper function

In [ ]:
from quiz_helper import get_pricing

## get pricing data into a dataframe

In [ ]:
returns_df = \
    get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        universe_end_date - pd.DateOffset(years=5),
        universe_end_date)\
    .pct_change()[1:].fillna(0) #convert prices into returns

returns_df

## Quiz 1

Check out the [numpy.cov documentation](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.cov.html).  Then think about what's wrong with the following use of numpy.cov

In [ ]:
# What's wrong with this?
annualization_factor = 252
covariance_assets_not_correct = annualization_factor*np.cov(returns_df)

In [ ]:
## TODO: Check the shape of the covariance matrix


## Answer 1 here:



## Quiz 2
How can you adjust the input so that we get the desired covariance matrix of assets?

In [ ]:
# TODO: calculate the covariance matrix of assets
annualization_factor = # ...
covariance_assets = # ...

In [ ]:
covariance_assets.shape

## Answer 2:

## Visualize the covariance matrix

In [ ]:
import seaborn as sns

In [ ]:
# view a heatmap of the covariance matrix
sns.heatmap(covariance_assets,cmap='Paired');
## If the colors aren't distinctive, please try a couple of these color schemes:
## cmap = 'tab10'
# cmap = 'Accent'

## Quiz 3
Looking at the colormap are covariances more likely to be positive or negative?  Are covariances likely to be above 0.10 or below 0.10?

## Answer 3 here:


## Fun Quiz!
Do you know what the [seaborn visualization package](https://seaborn.pydata.org/index.html) was named after?

## Fun Answer! here 
or just check the solution notebook!

## Solutions
The [solution notebook is here](historical_variance_solution.ipynb)